### Topics in Bioinformatics Human Evolution Part 2
Write a program that, for a given list of individuals:
1. determines the observed and expected frequency of heterozygotes at every SNP and calculates the difference
2. writes to a file:
- the observed heterozygosity for each SNP
- the expected heterozygosity for each SNP
- and the mean difference between observed and expected for each SNP
- The mean of each of these (obs,exp,diff) for the population

##### Loading files
- Load file
- Filters a VCF file to include only the specified samples using cyvcf2
- Save three new vcf files for each population

In [ ]:
# Load VCF
vcf_in = VariantFile("week_3_4/data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz")
vcf_header = str(vcf_in.header).split('\n')[-2] 

[E::idx_find_and_load] Could not retrieve index file for 'week_3_4/data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'


In [ ]:
# Filtering VCF file to contain the EAS, EUR, and YRI pops
from cyvcf2 import VCF, Writer

def filter_vcf_by_samples_cyvcf2(input_vcf, output_vcf, samples_to_keep):
    try:
        vcf = VCF(input_vcf)
        
        # Check if the requested samples are in the VCF file.
        # This is a good practice to avoid silent errors.
        vcf_samples = set(vcf.samples)
        requested_samples = set(samples_to_keep)
        missing_samples = requested_samples - vcf_samples
        if missing_samples:
            print(f"Warning: The following samples were not found in the VCF header: {', '.join(missing_samples)}")
        
        # Set the samples to be included. This is the fix for the TypeError.
        vcf.set_samples(samples_to_keep)
        
        w = Writer(output_vcf, vcf)
        for rec in vcf:
            w.write_record(rec)
        w.close()
        vcf.close()
        print(f"Successfully filtered VCF and saved to {output_vcf}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Implementation

# Load population panel
pops = pd.read_table("week_3_4/data/integrated_call_samples_v3.20130502.ALL.panel", sep="\t")

# Define samples for each population
YRI = list(set(pops[pops['pop'] == 'YRI']['sample']))
EAS = list(set(pops[pops['super_pop'] == 'EAS']['sample']))
EUR = list(set(pops[pops['super_pop'] == 'EUR']['sample']))

#Filter for each Population
filter_vcf_by_samples_cyvcf2(input_vcf_file, "week_3_4/data/filtered_YRI.vcf.gz", YRI)
filter_vcf_by_samples_cyvcf2(input_vcf_file, "week_3_4/data/filtered_EAS.vcf.gz", EAS)
filter_vcf_by_samples_cyvcf2(input_vcf_file, "week_3_4/data/filtered_EUR.vcf.gz", EUR)


Successfully filtered VCF and saved to week_3_4/data/filtered_EUR.vcf.gz


#### Function heterozygosity

Determines the observed and expected frequency of heterozygotes at every SNP and calculates the difference

In [22]:
from cyvcf2 import VCF
import numpy as np
import csv

def heterozygosity(input_vcf, output_het_stats):
    try:
        vcf_reader = VCF(input_vcf)
    
    # List to store differences for mean calculation
        ho_he_differences = []

        with open(output_het_stats, 'w', newline='') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t')
        
            # Write the header row
            writer.writerow(['CHROM', 'POS', 'REF', 'ALT', 'Ho (Observed Het)', 'He (Expected Het)', 'Ho - He Difference'])

            for variant in vcf_reader:
                # 1. Get Genotype Data
                gt_types = variant.gt_types # 0=hom-ref, 1=het, 2=hom-alt, 3=missing
            
                # 2. Identify non-missing genotypes
                non_missing_mask = (gt_types != 3)
                gt_types_valid = gt_types[non_missing_mask]
                num_non_missing_samples = np.sum(non_missing_mask)
            
                if num_non_missing_samples == 0:
                    continue # Skip site if all genotypes are missing

                # --------------------------------------------------------
                # a. Calculate Observed Heterozygosity ($H_o$)
                # Ho = (Number of heterozygous samples) / (Total number of non-missing samples)
                # --------------------------------------------------------
            
                # Count the number of '1's (heterozygous genotypes)
                num_hets = np.sum(gt_types_valid == 1)
                H_o = num_hets / num_non_missing_samples

                # --------------------------------------------------------
                # b. Calculate Expected Heterozygosity ($H_e$)
                # He = 2 * p * q, where p = AF (Allele Frequency) and q = 1 - p
                # --------------------------------------------------------
            
                # Calculate Allele Count (AC) and Allele Number (AN)
                ac = (np.sum(gt_types_valid == 1) * 1) + (np.sum(gt_types_valid == 2) * 2)
                an = num_non_missing_samples * 2
            
                p = ac / an # Allele Frequency (AF)
                q = 1 - p

                H_e = 2 * p * q

                # --------------------------------------------------------
                # c. Calculate Difference and Store for Mean
                # --------------------------------------------------------
                diff = H_o - H_e
                ho_he_differences.append(diff)

                # 3. Write results to the TSV file
                writer.writerow([
                    variant.CHROM,
                    variant.POS,
                    variant.REF,
                    variant.ALT[0],
                    f"{H_o:.6f}",
                    f"{H_e:.6f}",
                    f"{diff:.6f}"
                ])

        vcf_reader.close()
    
        # Calculate the mean difference across all SNPs
        mean_diff = np.mean(ho_he_differences) if ho_he_differences else 0

        print(f"Heterozygosity statistics saved to {output_het_stats}")
        print(f"\nMean Difference (Ho - He) across all SNPs: {mean_diff:.6f}")
        return mean_diff

    except Exception as e:
        print(f"An error occurred: {e}")

het_EUR = heterozygosity("week_3_4/data/filtered_YRI.vcf.gz", "week_3_4/data/filtered_EUR_Heterozygosity.tsv")
het_EAS = heterozygosity("week_3_4/data/filtered_EAS.vcf.gz", "week_3_4/data/filtered_EAS_Heterozygosity.tsv")
het_YRI = heterozygosity("week_3_4/data/filtered_YRI.vcf.gz", "week_3_4/data/filtered_YRI_Heterozygosity.tsv")

Heterozygosity statistics saved to week_3_4/data/filtered_EUR_Heterozygosity.tsv

Mean Difference (Ho - He) across all SNPs: 0.015414
Heterozygosity statistics saved to week_3_4/data/filtered_EAS_Heterozygosity.tsv

Mean Difference (Ho - He) across all SNPs: 0.013931
Heterozygosity statistics saved to week_3_4/data/filtered_YRI_Heterozygosity.tsv

Mean Difference (Ho - He) across all SNPs: 0.015414


In [ ]:
summary_output_file = "week_3_4/data/population_het_summary.tsv"
summary_data = [
    ("EUR", het_EUR),
    ("EAS", het_EAS),
    ("YRI", het_YRI)
]

try:
    with open(summary_output_file, 'w', newline='') as sumfile:
        summary_writer = csv.writer(sumfile, delimiter='\t')
        
        # Write header
        summary_writer.writerow(['Population', 'Mean_Ho_minus_He'])
        
        # Write data rows
        for pop, mean_diff in summary_data:
            summary_writer.writerow([pop, f"{mean_diff:.6f}"])

    print(f"\n All mean differences saved to single summary file: {summary_output_file}")
except Exception as e:
    print(f" Error writing summary file: {e}")


print("\n--- Summary of Mean Ho - He Difference ---")
print(f"EUR Mean Difference: {het_EUR:.6f}")
print(f"EAS Mean Difference: {het_EAS:.6f}")
print(f"YRI Mean Difference: {het_YRI:.6f}")



 All mean differences saved to single summary file: week_3_4/data/population_het_summary.tsv

--- Summary of Mean Ho - He Difference ---
EUR Mean Difference: 0.015414
EAS Mean Difference: 0.013931
YRI Mean Difference: 0.015414
